# Building Fast Queries on a CSV

We run an online laptop store and would like to build a way to answer a few different business questions about our inventory.

We will use the `laptops.csv` file as our inventory, which has been adapted from the original [dataset](https://www.kaggle.com/datasets/muhammetvarl/laptop-price).

Our dataset contains the following column titles:

- `ID`: A unique identifier for the laptop
- `Company`: The name of the company that produces the laptop
- `Product`: The name of the laptop
- `TypeName`: The type of laptop
- `Inches`: The size of the screen in inches
- `ScreenResolution`: The resolution of the screen
- `CPU`: The laptop CPU
- `RAM`: The amount of RAM in the laptop
- `Memory`: The size of the hard drive
- `GPU`: The graphics card name
- `OpSys`: The name of the operating system
- `Weight`: The laptop weight
- `Price`: The price of the laptop